# EEP/IAS 118 - Introductory Applied Econometrics
## Spring 2022, Villas-Boas
## MLR 4 and 5 Violation Examples


A common question we've received is "what does it look like if we violate our zero conditional mean (ML4) assumption? What about if we violate homoskedasticity (MLR5)?" 

This notebook walks through examples of data where one of these two assumptions is violated and shows what that looks like on both the scatterplots of y on x and y on the sample residuals.

In [ ]:
## Preamble - run this code to load our packages and set our plot theme
library(tidyverse)
plot_theme <- theme(
  panel.background = element_rect(fill = NA),  # change background color to white from grey
  axis.ticks = element_line(color = "grey95", size = 0.3), # make axis tick marks the same color as grid lines
  panel.grid.major = element_line(color = "grey95", size = 0.3), # change color of major grid lines (lines at displayed values)
  panel.grid.minor = element_line(color = "grey95", size = 0.3), #change color of minor grid lines (lines between displayed values)
  plot.caption = element_text(hjust = 0, face = "italic"), # left align bottom caption, make italic
  legend.key = element_blank(),
  axis.text=element_text(size=10, face="bold",),
  axis.title=element_text(size=16, face="bold",),
  legend.text=element_text(size=14),
  legend.title=element_text(size=16,  face="bold",),
  plot.title = element_text(size = 20,  face="bold", hjust=.5),
  plot.subtitle = element_text(size = 14, hjust=.5, margin=margin(2, 0, 5, 0)),
  legend.position = "none") # no legend

## 1. Violate Zero Conditional Mean (MLR4) but not Homoskedasticity (MLR5)

First we're going to generate some data where our population residuals are correlated with the values of our X variable but have the same variance across the space of X.

In [ ]:
# set random seed and generate some residuals
set.seed(12345)
sigma <- 2
u1 <- 20 + rnorm(20, sd= sigma) + 1*seq(1,20, by =1)
u2 <- -10 - rnorm(20, sd= sigma)+ 1*seq(1,20, by =1)
u3 <- 60 + rnorm(20, sd= sigma)+ 1*seq(1,20, by =1)
u <- c(u1, u2, u3)
# generate y where residual variance is constant but 
ex_df <- data.frame(x = 1:60,
                    u = u) %>%
         mutate(y = 1.5*x + u)

Now let's look at the scatterplot of Y on X along with the fitted regression line

In [ ]:
ggplot(ex_df, aes(y=y, x=x)) +
  geom_point() +
  geom_smooth(formula = y ~ x, se = FALSE, method = "lm") +
  plot_theme +
  labs(title = "Nonzero Conditional Residual Mean\n but Homoskedasticity")

We can see that there are three distinct groups of data. While they all share the same slope, they're shifted up or down relative to each other. As a result, we end up with only positive residuals when X is less than or equal to 20 or above 40, and only negative residuals if X is between 20 and 40. That means we've **violated MLR4:** given $X=18$, we know that we'll have a positive residual, and therefore $E[u~|~X=18] \neq 0$ and zero conditional mean assumption fails.

This pattern becomes even more clear if we plot the residuals:

In [ ]:
# run regression of y on x
reg <- lm(y ~ x, ex_df)

# Plot showing the pattern in the residuals
ex_df <- mutate(ex_df, res1 = reg$residuals)

ggplot(ex_df, aes(y=res1, x=x)) +
  geom_point() +
  geom_smooth(formula = y ~ x, se = FALSE, method = "lm") +
  plot_theme +
  labs(title = "Nonzero Conditional Residual Mean,\n Constant Error Variance",
       y = "Residuals",
       x = "X")

Now with our residuals on the y-axis we can see the pattern and MLR4 violation even more clearly. Note that our residuals are still **mean zero**, but when we condition on X they are no longer **zero conditional mean**.

But what about MLR5? Well in both of the plots we can see that the variance of our data points as well as the spread of the residuals are the same regardless of which value X takes on - while the mean value of the residuals shifts, the overall shape of the clouds are the same (which is good, since we generated them to be that way!). This means that homoskedasticity MLR5 still holds even though our conditional mean assumption failed.

## 2. Violate Homoskedasticity (MLR5) but not Zero Conditional Mean (MLR4)

Now let's take a look at the reverse situation from above, where we have error variance that's correlated with X (violation of MLR5) but no pattern in the expected value of our residuals (MLR4 holds).

In [ ]:
#regenerate data, replacing the previous version of the df
u1 <- rnorm(20, sd= 2)
u2 <- rnorm(20, sd= 8)
u3 <-rnorm(20, sd= 16)
u <- c(u1, u2, u3)
# generate y where residual variance is constant but 
ex_df <- data.frame(x = 1:60,
                    u = u) %>%
  mutate(y = 1.4*x + 20 + u)

# Plot Y on X along with the sample regression line
ggplot(ex_df, aes(y=y, x=x)) +
  geom_point() +
  geom_smooth(formula = y ~ x, se = FALSE, method = "lm") +
  plot_theme +
  labs(title = "Zero Conditional Residual Mean\n but Heteroskedasticity")

# run regression of y on x
reg <- lm(y ~ x, ex_df)

# Plot showing the pattern in the residuals
ex_df <- mutate(ex_df, res1 = reg$residuals)

ggplot(ex_df, aes(y=res1, x=x)) +
  geom_point() +
  geom_smooth(formula = y ~ x, se = FALSE, method = "lm") +
  plot_theme +
  labs(title = "Increasing Error Variance,\n Zero Conditional Mean",
       y = "Residuals",
       x = "X")


We can see here that the data becomes more disperse as X increases, though the slope stays the same. As a result, we end up with mean zero for our residuals **but** increasing variance. The giveaway of this is in the residual plot - we can see that the mean is zero across all values of X (flat blue line) but the residuals become much more spread out at higher values of X, indicating that the residual variance when $X \geq 40$ is a lot larger than when $X \leq 20$.